In [4]:
# Import packages
import os
import matplotlib.pyplot as plt

# Good practice to import non-standard packages after standard
import SintonFMT_LIB as S

In [5]:
# Get files
# select folder with files of interest. Example below
iv_dir = "occc"
iv_files = os.listdir(iv_dir)
# Printout for sanity check. You can comment it out
#print(iv_files)
iv_mfr = [ivf for ivf in iv_files if '.mfr' in ivf]
iv_txt = [ivf for ivf in iv_files if '.txt' in ivf]
#print(iv_mfr)
#print(iv_txt)

In [22]:
# Read data from files
# For the sake of this demo, I am only reading one file and going through each function individually. The functions in this mini-pipeline are combined in one function in SintonFMT_LIB
# Set values: rsh_v_cell does not change; if sun is set to a value, the analysis only returns results for that one intensity
iv_file = f"{iv_dir}/{iv_mfr[0]}"
rsh_v_cell=0.45
step=1
sun=None

# =========================== iv analysis broken up START =========================== #
# Load data and recipe file data from mfr file.
iv_data, mfi_contents = S.import_raw_data_from_file(iv_file)

# Correct data based on correction constants
corrected_data = S.correct_raw_data(iv_data, reference_constant=None,
                                      voltage_temperature_coefficient=None)

# Interpolation necessary to account for slight differences in intensity values at each flash point.
corrected_data = S.interpolate_load_data(corrected_data)

# Extracting I-V and pseudo-I-V data as a function of light intensity
corrected_data['iv_curve_intensity'] = S.get_iv_intensity_array(corrected_data,
                                                              step, sun)
corrected_data['suns_voc'] = S.get_suns_voc(corrected_data, step)
corrected_data['pseudo-iv_curve_intensity'] = S.get_piv_intensity_array(
    corrected_data['suns_voc'],
    corrected_data['iv_curve_intensity'])
corrected_data['intensity_array'] = corrected_data['iv_curve_intensity'][0, :, 2]

# Extracting performance versus intensity data
corrected_data = S.extract_parameter_v_intensity(corrected_data, rsh_v_cell)
# =========================== iv analysis broken up END =========================== #

# All of the above is condensed in one function:
corrected_data, mfi_contents = S.iv_analysis(mfr_file=iv_file, reference_constant=None,
                voltage_temperature_coefficient=None,
                rsh_v_cell=rsh_v_cell, step=step, sun=sun)


In [33]:
# Printouts and tips
# Some of the data are difficult to visualize. Below are some tips for slicing.

# Get the I-V curve at the first (i.e., minimum) measured light intensity. Columns are voltage, current, light intensity (all the same value)
#corrected_data['iv_curve_intensity'][:,0,:]

# Get the pseudo I-V curve at the first (i.e., minimum) measured light intensity
#corrected_data['pseudo-iv_curve_intensity'][:,0,:]

# Parameter versus intensity
#corrected_data['intensity_vmp']

# Suns-Voc data
#corrected_data['suns_voc']

# Keys of the dictionary that contain the types of information in corrected_data
#corrected_data.keys()
#print(corrected_data)
#print(mfi_contents)


The keys of interest for the corrected_data dictionary are: 

- **Measurement metadata**: 'reference_constant', 'voltage_temperature_coefficienct', 'temperature', 'max_intensity', 'min_intensity'
- **Processed data**: 'iv_curve_intensity', 'suns_voc', 'pseudo-iv_curve_intensity'
- **Performance characteristics**: 'intensity_pmp', 'intensity_imp', 'intensity_vmp', 'intensity_voc', 'intensity_isc', 'intensity_ff', 'intensity_module_efficiency', 'intensity_active_area_efficiency', 'intensity_rs', 'intensity_rsh', 'intensity_pseudo-pmp', 'intensity_pseudo-imp', 'intensity_pseudo-vmp', 'intensity_pseudo-voc', 'intensity_pseudo-isc', 'intensity_pseudo-ff', 'intensity_pseudo-module_efficiency', 'intensity_pseudo-active_area_efficiency'

In [32]:
# Looking at the text files on their own
# I-V data in text files are at 1000 W/m2.
iv_file = f"{iv_dir}/{iv_txt[0]}"
text_file_summary = S.import_suns_voc_data(iv_file)
lifetime_injection_data = text_file_summary['lifetime_injection_data']
iv_fit_data = text_file_summary['iv_fit_data']
vmp = corrected_data['intensity_vmp'][0]

lifetime_vmp = S.get_lifetime_at_vmp(lifetime_injection_data, iv_fit_data, vmp)
print(f"Carrier lifetime at Vmp is {lifetime_vmp*1e6:.1f} us at 1000 W/m2.")

Carrier lifetime at Vmp is 0.8 us at 1000 W/m2.
